In [2]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [3]:
import pandas as pd
import tensorflow as tf

# Training and Evaluation Data
Training and evaluation data should be provided in files already.

#### Check if train and eval files exists in /tmp/prod/
If not, please go back to ```Processing_ATL_JUNE.ipynb```

In [9]:
a_training_file = !ls /tmp/prod/atl_june_train-00001-of-*
a_training_file = a_training_file[0]
!wc -l $a_training_file

29801 /tmp/prod/atl_june_train-00001-of-00005


In [10]:
an_eval_file = !ls /tmp/prod/atl_june_eval-00001-of-*
an_eval_file = an_eval_file[0]
!wc -l $an_eval_file

11882 /tmp/prod/atl_june_eval-00001-of-00002


---
#### Have a look into the training data file

In [11]:
ALL_COLUMNS = [
    'YEAR', 'MONTH', 'DEP_DOW', 'AIRLINE', 
    'DEP_T', 'DEP', 'DEP_LAT', 'DEP_LON', 
    'MEAN_TEMP_DEP', 'MEAN_VIS_DEP', 'WND_SPD_DEP', 
    'ARR_DELAY', 'ARR', 'ARR_LAT', 'ARR_LON', 
    'MEAN_TEMP_ARR', 'MEAN_VIS_ARR', 'WND_SPD_ARR']

In [12]:
probe = pd.read_csv(a_training_file, names=ALL_COLUMNS)

In [13]:
probe.sample(frac=1.0)[:10]

,YEAR,MONTH,DEP_DOW,AIRLINE,DEP_T,DEP,DEP_LAT,DEP_LON,MEAN_TEMP_DEP,MEAN_VIS_DEP,WND_SPD_DEP,ARR_DELAY,ARR,ARR_LAT,ARR_LON,MEAN_TEMP_ARR,MEAN_VIS_ARR,WND_SPD_ARR
24063,2006,6,4,Delta Air Lines Inc.: DL,1220,ATL,33.63,-84.42,0.962185,0.968750,0.263566,4.0,SLC,40.78,-111.97,0.511774,0.009407,0.005301
833,2010,6,2,AirTran Airways Corporation: FL,845,ATL,33.63,-84.42,0.974790,1.000000,0.333333,0.0,RDU,35.87,-78.78,0.656201,0.008606,0.003000
2083,2005,6,3,ExpressJet Airlines Inc.: EV,1336,ATL,33.63,-84.42,0.567227,0.921875,0.294574,-3.0,RIC,37.50,-77.31,0.693878,0.009307,0.006501
17934,2008,6,5,Delta Air Lines Inc.: DL,1100,ATL,33.63,-84.42,0.789916,0.921875,0.348837,29.0,SEA,47.45,-122.31,0.290424,0.009407,0.006101
27489,2011,6,7,Delta Air Lines Inc.: DL,1110,ATL,33.63,-84.42,0.873950,0.921875,0.426357,-3.0,SRQ,27.39,-82.55,0.665620,0.009407,0.008001
12928,2006,6,3,ExpressJet Airlines Inc.: EV,1235,ATL,33.63,-84.42,0.319328,1.000000,0.426357,14.0,SAV,32.12,-81.20,0.508634,0.009407,0.007001
18109,2009,6,6,Delta Air Lines Inc.: DL,821,ATL,33.63,-84.42,0.848739,1.000000,0.372093,-16.0,SEA,47.45,-122.31,0.346939,0.008406,0.004900
1078,2011,6,7,Delta Air Lines Inc.: DL,1735,ATL,33.63,-84.42,0.873950,0.921875,0.426357,-20.0,RDU,35.87,-78.78,0.547881,0.009407,0.002600
25612,2006,6,6,Delta Air Lines Inc.: DL,900,ATL,33.63,-84.42,0.684874,0.875000,0.248062,-11.0,SNA,33.67,-117.86,0.530612,0.008906,0.005000
13675,2008,6,7,Delta Air Lines Inc.: DL,1455,ATL,33.63,-84.42,0.600840,0.859375,0.178295,1.0,SAV,32.12,-81.20,0.668760,0.008506,0.007301


In [14]:
probe.describe()

,YEAR,MONTH,DEP_DOW,DEP_T,DEP_LAT,DEP_LON,MEAN_TEMP_DEP,MEAN_VIS_DEP,WND_SPD_DEP,ARR_DELAY,ARR_LAT,ARR_LON,MEAN_TEMP_ARR,MEAN_VIS_ARR,WND_SPD_ARR
count,29801.000000,29801.0,29801.000000,29801.000000,29801.00,29801.00,29801.000000,29801.000000,29801.000000,29801.000000,29801.000000,29801.000000,29801.000000,29801.000000,29801.000000
mean,2007.119962,6.0,3.955740,1466.346767,33.63,-84.42,0.636172,0.855615,0.355867,12.971276,36.015935,-95.366662,0.547756,0.008777,0.006683
std,2.656701,0.0,1.974099,459.790567,0.00,0.00,0.183590,0.179274,0.140529,36.297191,5.352652,16.996090,0.138877,0.005830,0.002888
min,2002.000000,6.0,1.000000,655.000000,33.63,-84.42,0.000000,0.000000,0.000000,-50.000000,26.530000,-122.370000,0.141287,0.002702,0.000300
25%,2005.000000,6.0,2.000000,1028.000000,33.63,-84.42,0.504202,0.765625,0.255814,-7.000000,32.120000,-111.970000,0.433281,0.008506,0.004800
50%,2007.000000,6.0,4.000000,1455.000000,33.63,-84.42,0.634454,0.921875,0.348837,3.000000,37.500000,-90.370000,0.585557,0.009206,0.006101
75%,2009.000000,6.0,6.000000,1854.000000,33.63,-84.42,0.773109,0.984375,0.434109,18.000000,38.740000,-81.200000,0.660911,0.009407,0.008001
max,2011.000000,6.0,7.000000,2331.000000,33.63,-84.42,1.000000,1.000000,1.000000,593.000000,47.450000,-77.310000,0.822606,1.000000,0.024102


---
# ```tf.data``` Input Functions

In [15]:
print(list(probe.keys()))

['YEAR', 'MONTH', 'DEP_DOW', 'AIRLINE', 'DEP_T', 'DEP', 'DEP_LAT', 'DEP_LON', 'MEAN_TEMP_DEP', 'MEAN_VIS_DEP', 'WND_SPD_DEP', 'ARR_DELAY', 'ARR', 'ARR_LAT', 'ARR_LON', 'MEAN_TEMP_ARR', 'MEAN_VIS_ARR', 'WND_SPD_ARR']


In [16]:
DEFAULTS = [[0], [0], [0], ["none"], 
            [0], ["none"], [0.], [0.], 
            [0.], [0.], [0.], 
            [0.], ["none"], [0.], [0.], 
            [0.], [0.], [0.]]
SELECT = [1,2,4, 6, 7, 11, 12, 13, 14]

In [17]:
def make_input_fn(filename, mode, batch_size):
    def _input_fn():
        dataset = tf.data.TextLineDataset(filename)
        dataset = dataset.skip(1)

        def decode_csv(row):
            cols = tf.decode_csv(row, select_cols=SELECT, record_defaults=[DEFAULTS[i] for i in SELECT])
            features = dict(zip([ALL_COLUMNS[i] for i in SELECT], cols))
            return features

        def calc_hod(features):
            features['DEP_T'] = features['DEP_T'] // 100
            return features

        def pop_target(features):
            target = features.pop('ARR_DELAY')
            return features, target
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size=80000)
                
        dataset = (dataset.repeat()
                   .map(decode_csv)
                   .map(calc_hod)
                   .map(pop_target)
                   .batch(batch_size))

        return dataset.make_one_shot_iterator().get_next()
    
    return _input_fn

In [18]:
train_input_fn = make_input_fn(
    a_training_file, mode=tf.estimator.ModeKeys.TRAIN, batch_size=128)

eval_input_fn = make_input_fn(
    an_eval_file, mode=tf.estimator.ModeKeys.EVAL, batch_size=128)  

---
#### Verify that the input_functions return features and labels

In [19]:
try:
    sess.close()
except:
    pass
sess = tf.InteractiveSession()

In [20]:
input = train_input_fn()

In [21]:
sess.run(tf.global_variables_initializer())
sess.run(input)

({'ARR': array(['RIC', 'SRQ', 'SAN', 'SFO', 'SFO', 'SFO', 'STL', 'SJC', 'SNA',
         'SAV', 'SDF', 'RDU', 'SFO', 'RDU', 'SOP', 'STL', 'SHV', 'SAT',
         'SDF', 'SHV', 'SFO', 'SFO', 'RDU', 'SAT', 'SFO', 'ROC', 'RIC',
         'SFO', 'SRQ', 'RIC', 'RSW', 'RSW', 'SMF', 'SFO', 'SHV', 'STL',
         'SAV', 'SAN', 'SLC', 'SBN', 'RDU', 'SRQ', 'SEA', 'RSW', 'SGF',
         'STL', 'SDF', 'SDF', 'SDF', 'SEA', 'SAT', 'SBN', 'SAV', 'SDF',
         'SFO', 'SFO', 'ROA', 'SAT', 'SLC', 'STL', 'SHV', 'RDU', 'RIC',
         'RSW', 'SNA', 'SAN', 'SAV', 'RIC', 'SHV', 'ROC', 'SRQ', 'ROC',
         'ROC', 'SEA', 'SAV', 'SAT', 'SLC', 'STL', 'RIC', 'RIC', 'SAN',
         'RIC', 'SAV', 'STL', 'RDU', 'SAT', 'STL', 'SGF', 'SNA', 'SDF',
         'SBN', 'SAV', 'RIC', 'ROC', 'STL', 'SEA', 'SAT', 'SAN', 'SAT',
         'SNA', 'SLC', 'SAV', 'RIC', 'SNA', 'SAT', 'SAT', 'SBN', 'SRQ',
         'RSW', 'SAN', 'STL', 'SAT', 'SDF', 'RDU', 'SAN', 'SAV', 'SHV',
         'SDF', 'SRQ', 'RIC', 'SFO', 'SDF', 'RSW', 'STL',

---
# Feature engineering

In [22]:
arr_lat=tf.feature_column.numeric_column("ARR_LAT")
dep_lat=tf.feature_column.numeric_column("DEP_LAT")
arr_lon=tf.feature_column.numeric_column("ARR_LON")
dep_lon=tf.feature_column.numeric_column("DEP_LON")

dep_dow=tf.feature_column.categorical_column_with_identity("DEP_DOW", num_buckets=7)
dep_hod=tf.feature_column.categorical_column_with_identity("DEP_T", num_buckets=24)
dep_how=tf.feature_column.crossed_column(["DEP_T", "DEP_DOW"], 7*24)

dep_how_emb=tf.feature_column.embedding_column(dep_how, 10)

In [23]:
FEATURE_COLUMNS = [
    dep_how_emb,
    dep_lon,
    dep_lat,
    arr_lon,
    arr_lat
]

In [24]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column("DEP_HOW_EMB", shape=[10]),
    tf.feature_column.numeric_column("ARR_LON"),
    tf.feature_column.numeric_column("ARR_LAT"),
    tf.feature_column.numeric_column("DEP_LON"),
    tf.feature_column.numeric_column("DEP_LAT")
]

---
# A custom estimator from our own model

In [25]:
def model_fn ( features, labels, mode, params):

    if mode == tf.estimator.ModeKeys.EVAL:
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['feature_columns'])
        out = tf.layers.dense(input_layer, 1, activation=None)
        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)

        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss = loss,
            #eval_metric_ops={'my_metric': }
        )
    
    elif mode == tf.estimator.ModeKeys.TRAIN:     
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['feature_columns'])
        out = tf.layers.dense(input_layer, 1, activation=None)
        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)
        optimizer = tf.train.GradientDescentOptimizer(params['learning_rate'])
        train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

        return tf.estimator.EstimatorSpec(  
            mode,
            loss = loss,
            train_op = train_op)
    
    else:
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['input_columns'])
        out = tf.layers.dense(input_layer, 1, activation=None)
        return tf.estimator.EstimatorSpec(
            mode,
            predictions=out
            )
    

In [39]:
config = tf.estimator.RunConfig(model_dir="/tmp/model")

In [41]:
classifier = tf.estimator.Estimator(
        config=config,
        model_fn=model_fn,
        params={
            'feature_columns': FEATURE_COLUMNS,
            'input_columns': INPUT_COLUMNS,
            'learning_rate': 1e-6
        })

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f04b01ff050>, '_model_dir': '/tmp/model', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


---
# Training and Serving

In [42]:
def serving_input_fn():
  feature_placeholders = {
    'DEP_HOW_EMB' : tf.placeholder(tf.float32, [None]),
    'ARR_LON' : tf.placeholder(tf.float32, [None]),
    'ARR_LAT' : tf.placeholder(tf.float32, [None]),
    'DEP_LON' : tf.placeholder(tf.float32, [None]),
    'DEP_LAT' : tf.placeholder(tf.float32, [None]),
  }
  features = {
      key: tf.expand_dims(tensor, -1)
      for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

In [43]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=500)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps = 50, exporters=exporter)

#### At last, start the training!

In [29]:
tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpHt4E_t/model.ckpt.
INFO:tensorflow:loss = 1271.9917, step = 1
INFO:tensorflow:global_step/sec: 38.38
INFO:tensorflow:loss = 2313.6008, step = 101 (2.614 sec)
INFO:tensorflow:global_step/sec: 31.1
INFO:tensorflow:loss = 957.9072, step = 201 (3.213 sec)
INFO:tensorflow:global_step/sec: 30.529
INFO:tensorflow:loss = 2095.5754, step = 301 (3.273 sec)
INFO:tensorflow:global_step/sec: 37.1901
INFO:tensorflow:loss = 24

({'global_step': 500, 'loss': 1551.5258},
 ['/tmp/tmpHt4E_t/export/exporter/1541515586'])

---
#### Train with the full training set of ~300k records

In [33]:
all_train_files=!ls /tmp/prod/atl_june_train-*-of-00005
all_eval_files=!ls /tmp/prod/atl_june_eval-*-of-00002

In [44]:
full_train_input_fn = make_input_fn(
    all_train_files, mode=tf.estimator.ModeKeys.TRAIN, batch_size=512)

full_eval_input_fn = make_input_fn(
    all_eval_files, mode=tf.estimator.ModeKeys.EVAL, batch_size=512)  

train_spec = tf.estimator.TrainSpec(input_fn=full_train_input_fn, max_steps=2000)
eval_spec = tf.estimator.EvalSpec(input_fn=full_eval_input_fn, steps = 50, exporters=exporter)

In [45]:
tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/model/model.ckpt.
INFO:tensorflow:loss = 2831.354, step = 1
INFO:tensorflow:global_step/sec: 8.38259
INFO:tensorflow:loss = 1708.9395, step = 101 (11.933 sec)
INFO:tensorflow:global_step/sec: 8.0308
INFO:tensorflow:loss = 1270.7565, step = 201 (12.451 sec)
INFO:tensorflow:global_step/sec: 8.03686
INFO:tensorflow:loss = 1721.4529, step = 301 (12.446 sec)
INFO:tensorflow:global_step/sec: 7.57626
INFO:tensorflow:loss 

({'global_step': 2000, 'loss': 1654.5144},
 ['/tmp/model/export/exporter/1541516558'])